In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so


In [2]:
# Standard plotly imports
import plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
#tree classifier
import catboost as ctb 
from sklearn.ensemble import RandomForestClassifier as rf
#stacking
from sklearn.ensemble import StackingClassifier

import warnings
warnings.filterwarnings("ignore")

In [3]:
print("CatBoost version:", ctb.__version__)

CatBoost version: 0.24.2


In [4]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [5]:
print('# File sizes')
total_size = 0
start_path = '../input/jane-street-market-prediction'  # To get size of current directory
for path, dirs, files in os.walk(start_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)
print("Directory size: " + str(round(total_size/ 1000000, 2)) + 'MB')

# File sizes
Directory size: 6230.87MB


In [6]:
%%time
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')
features = pd.read_csv('../input/jane-street-market-prediction/features.csv')
example_test = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
sample_prediction_df = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
print ("Data is loaded!")

Data is loaded!
CPU times: user 1min 31s, sys: 8.46 s, total: 1min 40s
Wall time: 1min 51s


In [7]:
print('train shape is {}'.format(train.shape))
print('features shape is {}'.format(features.shape))
print('example_test shape is {}'.format(example_test.shape))
print('sample_prediction_df shape is {}'.format(sample_prediction_df.shape))

train shape is (2390491, 138)
features shape is (130, 30)
example_test shape is (15219, 133)
sample_prediction_df shape is (15219, 2)


In [8]:
train.head()

,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,feature_2,...,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id
0,0,0.000000,0.009916,0.014079,0.008773,0.001390,0.006270,1,-1.872746,-2.191242,...,NaN,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807,0
1,0,16.673515,-0.002828,-0.003226,-0.007319,-0.011114,-0.009792,-1,-1.349537,-1.704709,...,NaN,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684,1
2,0,0.000000,0.025134,0.027607,0.033406,0.034380,0.023970,-1,0.812780,-0.256156,...,NaN,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299,2
3,0,0.000000,-0.004730,-0.003273,-0.000461,-0.000476,-0.003200,-1,1.174378,0.344640,...,NaN,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469,3
4,0,0.138531,0.001252,0.002165,-0.001215,-0.006219,-0.002604,1,-3.172026,-3.093182,...,NaN,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633,4


In [9]:
missing_values_count = train.isnull().sum()
print (missing_values_count)
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

date              0
weight            0
resp_1            0
resp_2            0
resp_3            0
               ... 
feature_126    8853
feature_127    8853
feature_128    1921
feature_129    1921
ts_id             0
Length: 138, dtype: int64
% of missing data =  2.050000594444611


In [10]:
train = train[train['weight'] != 0]

train['action'] = ((train['weight'].values * train['resp'].values) > 0).astype('int')


X_train = train.loc[:, train.columns.str.contains('feature')]
y_train = train.loc[:, 'action']

In [11]:
X_train = X_train.fillna(-999)

In [12]:
x = train['action'].value_counts().index
y = train['action'].value_counts().values

trace2 = go.Bar(
     x=x ,
     y=y,
     marker=dict(
         color=y,
         colorscale = 'Viridis',
         reversescale = True
     ),
     name="Imbalance",    
 )
layout = dict(
     title="Data imbalance - action",
     #width = 900, height = 500,
     xaxis=go.layout.XAxis(
     automargin=True),
     yaxis=dict(
         showgrid=False,
         showline=False,
         showticklabels=True,
 #         domain=[0, 0.85],
     ), 
)
fig1 = go.Figure(data=[trace2], layout=layout)
iplot(fig1)


In [13]:
from sklearn.model_selection import train_test_split
 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state=2020)

In [14]:
del train, x, y

In [15]:
from catboost import CatBoostClassifier

In [16]:
model = CatBoostClassifier(iterations=500,
                          bagging_temperature=1,
                          random_strength=1,
                          thread_count=3,
                          l2_leaf_reg = 4.0, 
                          learning_rate = 0.07521709965938336,
                          save_snapshot=True,
                          snapshot_file='snapshot_best.bkp',
                          od_type='Iter',
                          od_wait=20,
                          random_state=2020,
                          custom_loss=['AUC', 'Accuracy']

                          )

In [17]:
%time model.fit(X_train, y_train)

0:	learn: 0.6929648	total: 2.6s	remaining: 21m 36s
1:	learn: 0.6927960	total: 5.03s	remaining: 20m 51s
2:	learn: 0.6926767	total: 7.4s	remaining: 20m 26s
3:	learn: 0.6925267	total: 9.53s	remaining: 19m 41s
4:	learn: 0.6923844	total: 11.6s	remaining: 19m 12s
5:	learn: 0.6922684	total: 13.5s	remaining: 18m 30s
6:	learn: 0.6921887	total: 15.4s	remaining: 18m 2s
7:	learn: 0.6921122	total: 17.6s	remaining: 18m 1s
8:	learn: 0.6920234	total: 19.9s	remaining: 18m 4s
9:	learn: 0.6919152	total: 21.8s	remaining: 17m 47s
10:	learn: 0.6918483	total: 23.5s	remaining: 17m 25s
11:	learn: 0.6917738	total: 25.5s	remaining: 17m 16s
12:	learn: 0.6917227	total: 27.1s	remaining: 16m 56s
13:	learn: 0.6916738	total: 30.1s	remaining: 17m 24s
14:	learn: 0.6915993	total: 32s	remaining: 17m 14s
15:	learn: 0.6915064	total: 33.5s	remaining: 16m 54s
16:	learn: 0.6914551	total: 35.4s	remaining: 16m 46s
17:	learn: 0.6913928	total: 37.3s	remaining: 16m 37s
18:	learn: 0.6913545	total: 39.5s	remaining: 16m 38s
19:	learn:

In [18]:
for (test_df, sample_prediction_df) in iter_test:
    X_test = test_df.loc[:, test_df.columns.str.contains('feature')]
    X_test.fillna(-999)
    y_preds = model.predict(X_test)
    sample_prediction_df.action = y_preds
    env.predict(sample_prediction_df)